In [0]:
month_or_year = "1_year_custom_joined"
test_pred_path = f"dbfs:/student-groups/Group_2_2/{month_or_year}/test_predictions"

In [0]:
test_pred_df = spark.read.parquet(test_pred_path)
print(f"test_pred_df has {test_pred_df.count()} rows")
display(test_pred_df.limit(10))

In [0]:
from pyspark.sql import functions as sf

test_pred_df = test_pred_df.withColumn(
    "true_class",
    sf.when(sf.col("DEP_DELAY_NEW") > 15, "Delayed").otherwise("On-Time")
).withColumn(
    "pred_class",
    sf.when(sf.col("prediction") > 15, "Delayed").otherwise("On-Time")
)

In [0]:
metrics = test_pred_df.groupBy("true_class", "pred_class").count().collect()
TP = TN = FP = FN = 0
for row in metrics:
    if row["true_class"] == "Delayed" and row["pred_class"] == "Delayed":
        TP = row["count"]
    elif row["true_class"] == "On-Time" and row["pred_class"] == "On-Time":
        TN = row["count"]
    elif row["true_class"] == "On-Time" and row["pred_class"] == "Delayed":
        FP = row["count"]
    elif row["true_class"] == "Delayed" and row["pred_class"] == "On-Time":
        FN = row["count"]

In [0]:
print(f"True Positives: {TP}")
print(f"True Negatives: {TN}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f2 = 5 * precision * recall / (4 * precision + recall)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F2: {f2}")